In [5]:
# Install required packages
!pip install rouge rouge-score nltk pandas

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=ea873c29d5d680e317eaf111ad1a5d88d66454b4255a3008ddca0cf256220552
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [11]:
import numpy as np
import pandas as pd
import tensorflow as tf
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Dot, Activation, Reshape, Concatenate, Embedding, LayerNormalization, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

# Download NLTK data
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import requests
import pandas as pd
import time
from datetime import datetime, timedelta
import json

# Konfigurasi API
API_KEY = "3ad8deb6cd0f4cfb9cb1747057f34e78"  # Gunakan API key yang Anda berikan
BASE_URL = "https://newsapi.org/v2/everything"

# Fungsi untuk mengecek API key dan koneksi
def test_api_key(api_key):
    params = {
        "q": "test",
        "pageSize": 1,
        "apiKey": api_key
    }

    try:
        response = requests.get(BASE_URL, params=params)
        data = response.json()

        if response.status_code == 200 and data.get("status") == "ok":
            print(f"✅ API Key valid. Status: {data.get('status')}")
            return True
        else:
            print(f"❌ API error: {data.get('message', 'Unknown error')}")
            return False
    except Exception as e:
        print(f"❌ Exception: {e}")
        return False

# Fungsi untuk mengambil artikel dengan paging
def fetch_all_articles(query, api_key, from_date=None, to_date=None, max_pages=10):
    all_articles = []
    total_available = 0

    for page in range(1, max_pages + 1):
        params = {
            "q": query,
            "pageSize": 100,  # Maksimum per halaman
            "page": page,
            "apiKey": api_key,
            "sortBy": "publishedAt"
        }

        if from_date:
            params["from"] = from_date
        if to_date:
            params["to"] = to_date

        print(f"Fetching page {page} for query '{query}'...")

        try:
            response = requests.get(BASE_URL, params=params)
            data = response.json()

            if response.status_code == 200 and data.get("status") == "ok":
                articles = data.get("articles", [])
                all_articles.extend(articles)

                # Simpan total hasil yang tersedia
                if page == 1:
                    total_available = data.get("totalResults", 0)
                    print(f"Total results available: {total_available}")

                print(f"Retrieved {len(articles)} articles on page {page}")

                # Jika kurang dari 100 artikel, berarti ini halaman terakhir
                if len(articles) < 100:
                    print(f"Reached last page with {len(articles)} articles")
                    break

                # Sleep untuk menghindari rate limiting
                time.sleep(0.5)
            else:
                print(f"Error on page {page}: {data.get('message', 'Unknown error')}")
                break
        except Exception as e:
            print(f"Exception on page {page}: {e}")
            break

    print(f"Total articles collected: {len(all_articles)} out of {total_available} available")
    return all_articles

# Cek API key
if not test_api_key(API_KEY):
    print("Please check your API key and try again.")
else:
    # Tanggal (30 hari terakhir)
    end_date = datetime.now().strftime("%Y-%m-%d")
    start_date = (datetime.now() - timedelta(days=30)).strftime("%Y-%m-%d")

    print(f"\nCollecting news articles from {start_date} to {end_date}")

    # Daftar query populer
    queries = [
        "technology",
        "politics", "business", "science", "health",
        "sports", "entertainment", "education", "environment", "finance",
        "medicine", "innovation", "artificial intelligence", "climate change", "space exploration",
        "bitcoin", "olympics", "COVID"
    ]

    # Kumpulkan artikel dari semua query
    all_data = []

    for query in queries:
        articles = fetch_all_articles(query, API_KEY, from_date=start_date, to_date=end_date, max_pages=5)

        # Tambahkan query sebagai tag
        for article in articles:
            article["query"] = query

        all_data.extend(articles)
        print(f"Collected {len(articles)} articles for '{query}'. Total: {len(all_data)}")

        # Cek rate limiting
        time.sleep(1)

    # Ubah ke DataFrame
    df = pd.DataFrame(all_data)

    # Hapus duplikat berdasarkan URL
    original_count = len(df)
    df = df.drop_duplicates(subset=["url"])
    print(f"Removed {original_count - len(df)} duplicate articles. Remaining: {len(df)}")

    # Periksa kolom apa saja yang tersedia
    print("\nColumns available in dataset:")
    print(df.columns.tolist())

    # Periksa konten artikel
    content_available = 'content' in df.columns
    if content_available:
        print(f"\nContent column available: {content_available}")
        print(f"Articles with non-null content: {df['content'].notna().sum()} of {len(df)}")

        # Tampilkan panjang konten
        if df['content'].notna().any():
            df['content_length'] = df['content'].fillna('').apply(len)
            print(f"Average content length: {df['content_length'].mean():.1f} characters")
            print(f"Min content length: {df['content_length'].min()} characters")
            print(f"Max content length: {df['content_length'].max()} characters")
    else:
        print("\nWARNING: 'content' column not available!")
        print("Using 'description' as content instead...")
        df['content'] = df['description']

    # Simpan hasil
    df.to_csv("newsapi_articles.csv", index=False)
    print(f"\nSaved {len(df)} articles to 'newsapi_articles.csv'")

    # Tampilkan beberapa contoh
    print("\nSample articles:")
    for i, row in df.sample(min(3, len(df))).iterrows():
        print(f"\n--- Article {i} ---")
        print(f"Title: {row.get('title', 'N/A')}")
        print(f"Source: {row.get('source', {}).get('name', 'N/A')}")
        print(f"Published: {row.get('publishedAt', 'N/A')}")
        content = row.get('content', 'N/A')
        if isinstance(content, str) and len(content) > 100:
            content = content[:100] + "..."
        print(f"Content: {content}")

# Setelah pengumpulan data, lanjutkan ke preprocessing
print("\nNow you can continue with preprocessing steps...")

In [12]:
# Load dataset
def load_data(file_path):
    df = pd.read_csv(file_path)
    # Select relevant columns: content (article text) and title
    df = df[['content', 'title']].dropna()  # Drop rows with missing content or title
    return df

# Clean text function
def clean_text(text):
    # Convert to string if not already
    text = str(text)
    # Remove special characters, URLs, and extra whitespace
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with single space
    text = text.lower().strip()  # Convert to lowercase and strip
    return text

# Tokenize and remove stopwords
def tokenize_text(text):
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    tokens = [token for token in tokens if token not in stop_words]
    return ' '.join(tokens)

# Preprocess dataset
def preprocess_data(df):
    # Clean content and title
    df['cleaned_content'] = df['content'].apply(clean_text).apply(tokenize_text)
    df['cleaned_title'] = df['title'].apply(clean_text).apply(tokenize_text)

    # Filter out empty strings after cleaning
    df = df[df['cleaned_content'] != '']
    df = df[df['cleaned_title'] != '']

    # Create input-output pairs
    input_texts = df['cleaned_content'].tolist()
    target_texts = df['cleaned_title'].tolist()

    return input_texts, target_texts

# Main function to process data
def main(file_path):
    # Load data
    df = load_data(file_path)

    # Preprocess data
    input_texts, target_texts = preprocess_data(df)

    # Split data into train and test sets
    train_inputs, test_inputs, train_targets, test_targets = train_test_split(
        input_texts, target_texts, test_size=0.2, random_state=42
    )

    # Save preprocessed data (optional)
    with open('train_inputs.txt', 'w', encoding='utf-8') as f:
        f.write('\n'.join(train_inputs))
    with open('train_targets.txt', 'w', encoding='utf-8') as f:
        f.write('\n'.join(train_targets))
    with open('test_inputs.txt', 'w', encoding='utf-8') as f:
        f.write('\n'.join(test_inputs))
    with open('test_targets.txt', 'w', encoding='utf-8') as f:
        f.write('\n'.join(test_targets))

    print(f"Total samples: {len(input_texts)}")
    print(f"Training samples: {len(train_inputs)}")
    print(f"Test samples: {len(test_inputs)}")

    # Example of input-output pair
    print("\nExample Input-Output Pair:")
    print(f"Input (Article): {train_inputs[0][:200]}...")
    print(f"Output (Title): {train_targets[0]}")

# Run the main function
if __name__ == "__main__":
    file_path = 'newsapi_articles.csv'  # Adjust path if needed
    main(file_path)

Total samples: 1327
Training samples: 1061
Test samples: 266

Example Input-Output Pair:
Input (Article): shannon sharpe pro football hall famer espn analyst grappling serious allegations second sexual assault lawsuit surfaced time different accuser according 2307 chars...
Output (Title): shannon sharpe hit second sexual assault lawsuit new accuser


In [14]:

# Parameters
MAX_VOCAB_SIZE = 20000  # Maximum vocabulary size
EMBEDDING_DIM = 128     # Embedding dimension
LSTM_UNITS = 256        # Number of LSTM units
MAX_ARTICLE_LEN = 200   # Maximum length of article (input)
MAX_TITLE_LEN = 20      # Maximum length of title (output)

# Load preprocessed data
def load_preprocessed_data(train_input_path, train_target_path, test_input_path, test_target_path):
    with open(train_input_path, 'r', encoding='utf-8') as f:
        train_inputs = f.read().splitlines()
    with open(train_target_path, 'r', encoding='utf-8') as f:
        train_targets = f.read().splitlines()
    with open(test_input_path, 'r', encoding='utf-8') as f:
        test_inputs = f.read().splitlines()
    with open(test_target_path, 'r', encoding='utf-8') as f:
        test_targets = f.read().splitlines()
    return train_inputs, train_targets, test_inputs, test_targets

# Tokenize and pad sequences
def prepare_sequences(input_texts, target_texts, max_input_len, max_target_len):
    # Tokenize input (articles)
    input_tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, oov_token='<OOV>')
    input_tokenizer.fit_on_texts(input_texts)
    input_sequences = input_tokenizer.texts_to_sequences(input_texts)
    input_padded = pad_sequences(input_sequences, maxlen=max_input_len, padding='post', truncating='post')

    # Tokenize target (titles)
    target_tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, oov_token='<OOV>')
    target_tokenizer.fit_on_texts(target_texts)
    target_sequences = target_tokenizer.texts_to_sequences(target_texts)
    target_padded = pad_sequences(target_sequences, maxlen=max_target_len, padding='post', truncating='post')

    # Prepare decoder input and output (shifted by one for teacher forcing)
    decoder_input_data = target_padded[:, :-1]
    decoder_target_data = target_padded[:, 1:]

    return input_padded, decoder_input_data, decoder_target_data, input_tokenizer, target_tokenizer

# Build LSTM encoder-decoder model
def build_model(input_vocab_size, target_vocab_size, embedding_dim, lstm_units, max_target_len):
    # Encoder
    encoder_inputs = Input(shape=(None,))
    encoder_embedding = Embedding(input_vocab_size, embedding_dim)(encoder_inputs)
    encoder_lstm, state_h, state_c = LSTM(lstm_units, return_state=True)(encoder_embedding)
    encoder_states = [state_h, state_c]

    # Decoder
    decoder_inputs = Input(shape=(None,))
    decoder_embedding = Embedding(target_vocab_size, embedding_dim)(decoder_inputs)
    decoder_lstm = LSTM(lstm_units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
    decoder_dense = Dense(target_vocab_size, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)

    # Model
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

# Main function
def main():
    # Load preprocessed data
    train_inputs, train_targets, test_inputs, test_targets = load_preprocessed_data(
        'train_inputs.txt', 'train_targets.txt', 'test_inputs.txt', 'test_targets.txt'
    )

    # Prepare sequences
    input_padded, decoder_input_data, decoder_target_data, input_tokenizer, target_tokenizer = prepare_sequences(
        train_inputs + test_inputs, train_targets + test_targets, MAX_ARTICLE_LEN, MAX_TITLE_LEN
    )

    # Split data back into train and test
    train_inputs_padded, test_inputs_padded, train_decoder_inputs, test_decoder_inputs, train_decoder_targets, test_decoder_targets = train_test_split(
        input_padded, decoder_input_data, decoder_target_data, test_size=0.2, random_state=42
    )

    # Get vocabulary sizes
    input_vocab_size = min(len(input_tokenizer.word_index) + 1, MAX_VOCAB_SIZE)
    target_vocab_size = min(len(target_tokenizer.word_index) + 1, MAX_VOCAB_SIZE)

    # Build and train model
    model = build_model(input_vocab_size, target_vocab_size, EMBEDDING_DIM, LSTM_UNITS, MAX_TITLE_LEN - 1)
    model.summary()

    # Train the model
    model.fit(
        [train_inputs_padded, train_decoder_inputs],
        train_decoder_targets,
        batch_size=64,
        epochs=10,
        validation_data=([test_inputs_padded, test_decoder_inputs], test_decoder_targets)
    )

    # Save the model
    model.save('lstm_encoder_decoder_model.h5')

    # Save tokenizers (optional)
    import pickle
    with open('input_tokenizer.pkl', 'wb') as f:
        pickle.dump(input_tokenizer, f)
    with open('target_tokenizer.pkl', 'wb') as f:
        pickle.dump(target_tokenizer, f)

    print("Model training completed and saved.")

if __name__ == "__main__":
    main()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, None, 128) │  1,479,296 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, None, 128) │    778,624 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 256),     │    394,240 │ embedding[0][0]   │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, None,     │    394,240 │ embedding_1[0][0… │
│                     │ 256), (None,      │            │ lstm[0][1],       │
│                     │ 256), (None,      │            │ lstm[0][2]        │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, None,      │  1,563,331 │ lstm_1[0][0]      │
│                     │ 6083)             │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,609,731 (17.58 MB)

 Trainable params: 4,609,731 (17.58 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 46s 2s/step - accuracy: 0.4634 - loss: 7.9371 - val_accuracy: 0.5708 - val_loss: 4.3166
Epoch 2/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 81s 2s/step - accuracy: 0.5764 - loss: 4.0585 - val_accuracy: 0.5708 - val_loss: 4.2054
Epoch 3/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 85s 3s/step - accuracy: 0.5764 - loss: 3.7219 - val_accuracy: 0.5708 - val_loss: 4.1851
Epoch 4/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 45s 3s/step - accuracy: 0.5764 - loss: 3.6078 - val_accuracy: 0.5714 - val_loss: 4.1816
Epoch 5/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 82s 3s/step - accuracy: 0.5766 - loss: 3.5439 - val_accuracy: 0.5712 - val_loss: 4.1999
Epoch 6/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 81s 3s/step - accuracy: 0.5764 - loss: 3.5034 - val_accuracy: 0.5706 - val_loss: 4.2378
Epoch 7/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 42s 2s/step - accuracy: 0.5764 - loss: 3.4717 - val_accuracy: 0.5702 - val_loss: 4.2649
Epoch 8/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 83s 3s/step - accuracy: 0.5763 - loss: 3.4409 - val_accuracy: 0.5700 - val_loss:

Model training completed and saved.


In [30]:
# Parameters
MAX_VOCAB_SIZE = 20000  # Maximum vocabulary size
EMBEDDING_DIM = 128     # Embedding dimension
ATTENTION_UNITS = 256   # Number of units for attention
MAX_ARTICLE_LEN = 200   # Maximum length of article (input)
MAX_TITLE_LEN = 20      # Maximum length of title (output)

# Load preprocessed data
def load_preprocessed_data(train_input_path, train_target_path, test_input_path, test_target_path):
    with open(train_input_path, 'r', encoding='utf-8') as f:
        train_inputs = f.read().splitlines()
    with open(train_target_path, 'r', encoding='utf-8') as f:
        train_targets = f.read().splitlines()
    with open(test_input_path, 'r', encoding='utf-8') as f:
        test_inputs = f.read().splitlines()
    with open(test_target_path, 'r', encoding='utf-8') as f:
        test_targets = f.read().splitlines()
    return train_inputs, train_targets, test_inputs, test_targets

# Tokenize and pad sequences
def prepare_sequences(input_texts, target_texts, max_input_len, max_target_len):
    # Tokenize input (articles)
    input_tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, oov_token='<OOV>')
    input_tokenizer.fit_on_texts(input_texts)
    input_sequences = input_tokenizer.texts_to_sequences(input_texts)
    input_padded = pad_sequences(input_sequences, maxlen=max_input_len, padding='post', truncating='post')

    # Tokenize target (titles)
    target_tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, oov_token='<OOV>')
    target_tokenizer.fit_on_texts(target_texts)
    target_sequences = target_tokenizer.texts_to_sequences(target_texts)
    target_padded = pad_sequences(target_sequences, maxlen=max_target_len, padding='post', truncating='post')

    # Prepare decoder input and output (shifted by one for teacher forcing)
    decoder_input_data = target_padded[:, :-1]
    decoder_target_data = target_padded[:, 1:]

    return input_padded, decoder_input_data, decoder_target_data, input_tokenizer, target_tokenizer

# Build encoder-decoder with attention
def build_attention_model(input_vocab_size, target_vocab_size, embedding_dim, attention_units, max_target_len):
    # Encoder
    encoder_inputs = Input(shape=(None,))
    encoder_embedding = Embedding(input_vocab_size, embedding_dim)(encoder_inputs)
    encoder_dense = Dense(attention_units, activation='relu')(encoder_embedding)  # Dense layer for encoding

    # Decoder
    decoder_inputs = Input(shape=(None,))
    decoder_embedding = Embedding(target_vocab_size, embedding_dim)(decoder_inputs)
    decoder_dense = Dense(attention_units, activation='relu')(decoder_embedding)  # Dense layer for decoding

    # Attention mechanism (Dot Product Attention)
    attention_score = Dot(axes=(2, 2))([decoder_dense, encoder_dense])  # Compute alignment scores
    attention_weights = Activation('softmax')(attention_score)  # Normalize scores
    context_vector = Dot(axes=(2, 1))([attention_weights, encoder_dense])  # Weighted sum of encoder outputs

    # Combine context vector with decoder output
    decoder_combined = Concatenate(axis=-1)([context_vector, decoder_dense])
    decoder_output = Dense(attention_units, activation='relu')(decoder_combined)
    decoder_final = Dense(target_vocab_size, activation='softmax')(decoder_output)

    # Model
    model = Model([encoder_inputs, decoder_inputs], decoder_final)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

# Main function
def main():
    # Load preprocessed data
    train_inputs, train_targets, test_inputs, test_targets = load_preprocessed_data(
        'train_inputs.txt', 'train_targets.txt', 'test_inputs.txt', 'test_targets.txt'
    )

    # Prepare sequences
    input_padded, decoder_input_data, decoder_target_data, input_tokenizer, target_tokenizer = prepare_sequences(
        train_inputs + test_inputs, train_targets + test_targets, MAX_ARTICLE_LEN, MAX_TITLE_LEN
    )

    # Split data back into train and test
    train_inputs_padded, test_inputs_padded, train_decoder_inputs, test_decoder_inputs, train_decoder_targets, test_decoder_targets = train_test_split(
        input_padded, decoder_input_data, decoder_target_data, test_size=0.2, random_state=42
    )

    # Get vocabulary sizes
    input_vocab_size = min(len(input_tokenizer.word_index) + 1, MAX_VOCAB_SIZE)
    target_vocab_size = min(len(target_tokenizer.word_index) + 1, MAX_VOCAB_SIZE)

    # Build and train model
    model = build_attention_model(input_vocab_size, target_vocab_size, EMBEDDING_DIM, ATTENTION_UNITS, MAX_TITLE_LEN - 1)
    model.summary()

    # Train the model
    model.fit(
        [train_inputs_padded, train_decoder_inputs],
        train_decoder_targets,
        batch_size=64,
        epochs=10,
        validation_data=([test_inputs_padded, test_decoder_inputs], test_decoder_targets)
    )

    # Save the model
    model.save('encoder_decoder_attention_model.h5')

    # Save tokenizers
    with open('input_tokenizer.pkl', 'wb') as f:
        pickle.dump(input_tokenizer, f)
    with open('target_tokenizer.pkl', 'wb') as f:
        pickle.dump(target_tokenizer, f)

    print("Model training completed and saved.")

if __name__ == "__main__":
    main()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_23      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_22      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_23        │ (None, None, 128) │    778,624 │ input_layer_23[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_22        │ (None, None, 128) │  1,479,296 │ input_layer_22[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_26 (Dense)    │ (None, None, 256) │     33,024 │ embedding_23[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_25 (Dense)    │ (None, None, 256) │     33,024 │ embedding_22[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot (Dot)           │ (None, None,      │          0 │ dense_26[0][0],   │
│                     │ None)             │            │ dense_25[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, None,      │          0 │ dot[0][0]         │
│ (Activation)        │ None)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot_1 (Dot)         │ (None, None, 256) │          0 │ activation[0][0], │
│                     │                   │            │ dense_25[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, None, 512) │          0 │ dot_1[0][0],      │
│ (Concatenate)       │                   │            │ dense_26[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_27 (Dense)    │ (None, None, 256) │    131,328 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_28 (Dense)    │ (None, None,      │  1,563,331 │ dense_27[0][0]    │
│                     │ 6083)             │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,018,627 (15.33 MB)

 Trainable params: 4,018,627 (15.33 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.4639 - loss: 7.8023 - val_accuracy: 0.5708 - val_loss: 4.5687
Epoch 2/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - accuracy: 0.5764 - loss: 3.9037 - val_accuracy: 0.5708 - val_loss: 4.1017
Epoch 3/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.5764 - loss: 3.6825 - val_accuracy: 0.5708 - val_loss: 4.4477
Epoch 4/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 43s 1s/step - accuracy: 0.5764 - loss: 3.6052 - val_accuracy: 0.5708 - val_loss: 4.3373
Epoch 5/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - accuracy: 0.5764 - loss: 3.5553 - val_accuracy: 0.5708 - val_loss: 4.3771
Epoch 6/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.5764 - loss: 3.5288 - val_accuracy: 0.5708 - val_loss: 4.3774
Epoch 7/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - accuracy: 0.5764 - loss: 3.4757 - val_accuracy: 0.5708 - val_loss: 4.4549
Epoch 8/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.5764 - loss: 3.4337 - val_accuracy: 0.5708 - val_loss:

Model training completed and saved.


In [33]:
# Parameters
MAX_VOCAB_SIZE = 20000  # Maximum vocabulary size
EMBEDDING_DIM = 128     # Embedding dimension
NUM_HEADS = 4           # Number of attention heads
FF_DIM = 512            # Feed-forward network dimension
NUM_LAYERS = 2          # Number of transformer layers
DROPOUT_RATE = 0.1      # Dropout rate
MAX_ARTICLE_LEN = 200   # Maximum length of article (input)
MAX_TITLE_LEN = 20      # Maximum length of title (output)

# Multi-Head Attention Layer
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        assert d_model % num_heads == 0
        self.depth = d_model // num_heads

        self.wq = Dense(d_model)
        self.wk = Dense(d_model)
        self.wv = Dense(d_model)
        self.dense = Dense(d_model)

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, q, k, v, mask=None):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)
        k = self.wk(k)
        v = self.wv(v)

        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)

        scaled_attention = tf.matmul(q, k, transpose_b=True) / tf.sqrt(tf.cast(self.depth, tf.float32))
        if mask is not None:
            scaled_attention += (mask * -1e9)
        attention_weights = tf.nn.softmax(scaled_attention, axis=-1)
        output = tf.matmul(attention_weights, v)

        output = tf.transpose(output, perm=[0, 2, 1, 3])
        output = tf.reshape(output, (batch_size, -1, self.d_model))
        return self.dense(output)

# Transformer Encoder Layer
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, ff_dim, dropout_rate=0.1):
        super(EncoderLayer, self).__init__()
        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation='relu'),
            Dense(d_model)
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(dropout_rate)
        self.dropout2 = Dropout(dropout_rate)

    def call(self, x, training, mask=None):
        attn_output = self.mha(x, x, x, mask)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

# Transformer Decoder Layer
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, ff_dim, dropout_rate=0.1):
        super(DecoderLayer, self).__init__()
        self.mha1 = MultiHeadAttention(d_model, num_heads)
        self.mha2 = MultiHeadAttention(d_model, num_heads)
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation='relu'),
            Dense(d_model)
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.layernorm3 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(dropout_rate)
        self.dropout2 = Dropout(dropout_rate)
        self.dropout3 = Dropout(dropout_rate)

    def call(self, x, enc_output, training, look_ahead_mask=None, padding_mask=None):
        attn1 = self.mha1(x, x, x, look_ahead_mask)
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(x + attn1)

        attn2 = self.mha2(out1, enc_output, enc_output, padding_mask)
        attn2 = self.dropout2(attn2, training=training)
        out2 = self.layernorm2(out1 + attn2)

        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        return self.layernorm3(out2 + ffn_output)

# Transformer Model
def build_transformer_model(input_vocab_size, target_vocab_size, d_model, num_heads, ff_dim, num_layers, max_target_len, dropout_rate=0.1):
    # Encoder
    encoder_inputs = Input(shape=(None,))
    enc_embedding = Embedding(input_vocab_size, d_model)(encoder_inputs)
    enc_output = enc_embedding
    for _ in range(num_layers):
        enc_output = EncoderLayer(d_model, num_heads, ff_dim, dropout_rate)(enc_output, training=True)

    # Decoder
    decoder_inputs = Input(shape=(None,))
    dec_embedding = Embedding(target_vocab_size, d_model)(decoder_inputs)
    dec_output = dec_embedding
    for _ in range(num_layers):
        dec_output = DecoderLayer(d_model, num_heads, ff_dim, dropout_rate)(
            dec_output, enc_output, training=True, look_ahead_mask=None, padding_mask=None
        )

    # Output layer
    outputs = Dense(target_vocab_size, activation='softmax')(dec_output)

    # Model
    model = Model([encoder_inputs, decoder_inputs], outputs)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Load preprocessed data
def load_preprocessed_data(train_input_path, train_target_path, test_input_path, test_target_path):
    with open(train_input_path, 'r', encoding='utf-8') as f:
        train_inputs = f.read().splitlines()
    with open(train_target_path, 'r', encoding='utf-8') as f:
        train_targets = f.read().splitlines()
    with open(test_input_path, 'r', encoding='utf-8') as f:
        test_inputs = f.read().splitlines()
    with open(test_target_path, 'r', encoding='utf-8') as f:
        test_targets = f.read().splitlines()
    return train_inputs, train_targets, test_inputs, test_targets

# Tokenize and pad sequences
def prepare_sequences(input_texts, target_texts, max_input_len, max_target_len):
    input_tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, oov_token='<OOV>')
    input_tokenizer.fit_on_texts(input_texts)
    input_sequences = input_tokenizer.texts_to_sequences(input_texts)
    input_padded = pad_sequences(input_sequences, maxlen=max_input_len, padding='post', truncating='post')

    target_tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, oov_token='<OOV>')
    target_tokenizer.fit_on_texts(target_texts)
    target_sequences = target_tokenizer.texts_to_sequences(target_texts)
    target_padded = pad_sequences(target_sequences, maxlen=max_target_len, padding='post', truncating='post')

    decoder_input_data = target_padded[:, :-1]
    decoder_target_data = target_padded[:, 1:]

    return input_padded, decoder_input_data, decoder_target_data, input_tokenizer, target_tokenizer

# Main function
def main():
    # Load preprocessed data
    train_inputs, train_targets, test_inputs, test_targets = load_preprocessed_data(
        'train_inputs.txt', 'train_targets.txt', 'test_inputs.txt', 'test_targets.txt'
    )

    # Prepare sequences
    input_padded, decoder_input_data, decoder_target_data, input_tokenizer, target_tokenizer = prepare_sequences(
        train_inputs + test_inputs, train_targets + test_targets, MAX_ARTICLE_LEN, MAX_TITLE_LEN
    )

    # Split data back into train and test
    train_inputs_padded, test_inputs_padded, train_decoder_inputs, test_decoder_inputs, train_decoder_targets, test_decoder_targets = train_test_split(
        input_padded, decoder_input_data, decoder_target_data, test_size=0.2, random_state=42
    )

    # Get vocabulary sizes
    input_vocab_size = min(len(input_tokenizer.word_index) + 1, MAX_VOCAB_SIZE)
    target_vocab_size = min(len(target_tokenizer.word_index) + 1, MAX_VOCAB_SIZE)

    # Build and train model
    model = build_transformer_model(
        input_vocab_size, target_vocab_size, EMBEDDING_DIM, NUM_HEADS, FF_DIM, NUM_LAYERS, MAX_TITLE_LEN - 1, DROPOUT_RATE
    )
    model.summary()

    # Train the model
    model.fit(
        [train_inputs_padded, train_decoder_inputs],
        train_decoder_targets,
        batch_size=64,
        epochs=10,
        validation_data=([test_inputs_padded, test_decoder_inputs], test_decoder_targets)
    )

    # Save the model
    model.save('transformer_model.h5')

    # Save tokenizers
    with open('input_tokenizer.pkl', 'wb') as f:
        pickle.dump(input_tokenizer, f)
    with open('target_tokenizer.pkl', 'wb') as f:
        pickle.dump(target_tokenizer, f)

    print("Model training completed and saved.")

if __name__ == "__main__":
    main()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_25      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_25        │ (None, None, 128) │  1,479,296 │ input_layer_25[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_28      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_layer       │ (None, None, 128) │    198,272 │ embedding_25[0][… │
│ (EncoderLayer)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_26        │ (None, None, 128) │    778,624 │ input_layer_28[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_layer_1     │ (None, None, 128) │    198,272 │ encoder_layer[0]… │
│ (EncoderLayer)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_layer       │ (None, None, 128) │    264,576 │ embedding_26[0][… │
│ (DecoderLayer)      │                   │            │ encoder_layer_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_layer_1     │ (None, None, 128) │    264,576 │ decoder_layer[0]… │
│ (DecoderLayer)      │                   │            │ encoder_layer_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_61 (Dense)    │ (None, None,      │    784,707 │ decoder_layer_1[… │
│                     │ 6083)             │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,968,323 (15.14 MB)

 Trainable params: 3,968,323 (15.14 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 82s 4s/step - accuracy: 0.4640 - loss: 7.4619 - val_accuracy: 0.5708 - val_loss: 5.8932
Epoch 2/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 50s 3s/step - accuracy: 0.5764 - loss: 5.3445 - val_accuracy: 0.5708 - val_loss: 4.6094
Epoch 3/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 52s 3s/step - accuracy: 0.5764 - loss: 4.2004 - val_accuracy: 0.5708 - val_loss: 4.5063
Epoch 4/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 79s 3s/step - accuracy: 0.5764 - loss: 3.9723 - val_accuracy: 0.5708 - val_loss: 4.1491
Epoch 5/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 85s 3s/step - accuracy: 0.5764 - loss: 3.6965 - val_accuracy: 0.5708 - val_loss: 4.0809
Epoch 6/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 51s 3s/step - accuracy: 0.5764 - loss: 3.6393 - val_accuracy: 0.5708 - val_loss: 4.2215
Epoch 7/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 77s 3s/step - accuracy: 0.5768 - loss: 3.6389 - val_accuracy: 0.5708 - val_loss: 4.1754
Epoch 8/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 87s 3s/step - accuracy: 0.5764 - loss: 3.5692 - val_accuracy: 0.5708 - val_loss:

Model training completed and saved.


In [40]:
# Parameters (sesuaikan dengan pelatihan)
MAX_VOCAB_SIZE = 20000
MAX_ARTICLE_LEN = 200
MAX_TITLE_LEN = 20
EMBEDDING_DIM = 128
LSTM_UNITS = 256
ATTENTION_UNITS = 256
NUM_HEADS = 4
FF_DIM = 512
NUM_LAYERS = 2
DROPOUT_RATE = 0.1

# Clean text function (sama seperti preprocessing)
def clean_text(text):
    text = str(text)
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces
    text = text.lower().strip()
    return text

# Tokenize and remove stopwords
def tokenize_text(text):
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    tokens = [token for token in tokens if token not in stop_words]
    return ' '.join(tokens)

# Preprocess article
def preprocess_article(article, input_tokenizer):
    cleaned_article = clean_text(article)
    tokenized_article = tokenize_text(cleaned_article)
    sequence = input_tokenizer.texts_to_sequences([tokenized_article])
    padded_sequence = pad_sequences(sequence, maxlen=MAX_ARTICLE_LEN, padding='post', truncating='post')
    return padded_sequence

# Definisikan kembali kelas kustom untuk Transformer
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, **kwargs):
        super(MultiHeadAttention, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.d_model = d_model
        assert d_model % num_heads == 0
        self.depth = d_model // num_heads
        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)
        self.dense = tf.keras.layers.Dense(d_model)

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, q, k, v, mask=None):
        batch_size = tf.shape(q)[0]
        q = self.wq(q)
        k = self.wk(k)
        v = self.wv(v)
        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)
        scaled_attention = tf.matmul(q, k, transpose_b=True) / tf.sqrt(tf.cast(self.depth, tf.float32))
        if mask is not None:
            scaled_attention += (mask * -1e9)
        attention_weights = tf.nn.softmax(scaled_attention, axis=-1)
        output = tf.matmul(attention_weights, v)
        output = tf.transpose(output, perm=[0, 2, 1, 3])
        output = tf.reshape(output, (batch_size, -1, self.d_model))
        return self.dense(output)

    def get_config(self):
        config = super(MultiHeadAttention, self).get_config()
        config.update({'d_model': self.d_model, 'num_heads': self.num_heads})
        return config

class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, ff_dim, dropout_rate=0.1, **kwargs):
        super(EncoderLayer, self).__init__(**kwargs)
        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = tf.keras.Sequential([
            tf.keras.layers.Dense(ff_dim, activation='relu'),
            tf.keras.layers.Dense(d_model)
        ])
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
        self.dropout2 = tf.keras.layers.Dropout(dropout_rate)

    def call(self, x, training, mask=None):
        attn_output = self.mha(x, x, x, mask)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

    def get_config(self):
        config = super(EncoderLayer, self).get_config()
        config.update({
            'd_model': self.mha.d_model,
            'num_heads': self.mha.num_heads,
            'ff_dim': self.ffn.layers[0].units,
            'dropout_rate': self.dropout1.rate
        })
        return config

class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, ff_dim, dropout_rate=0.1, **kwargs):
        super(DecoderLayer, self).__init__(**kwargs)
        self.mha1 = MultiHeadAttention(d_model, num_heads)
        self.mha2 = MultiHeadAttention(d_model, num_heads)
        self.ffn = tf.keras.Sequential([
            tf.keras.layers.Dense(ff_dim, activation='relu'),
            tf.keras.layers.Dense(d_model)
        ])
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
        self.dropout2 = tf.keras.layers.Dropout(dropout_rate)
        self.dropout3 = tf.keras.layers.Dropout(dropout_rate)

    def call(self, x, enc_output, training, look_ahead_mask=None, padding_mask=None):
        attn1 = self.mha1(x, x, x, look_ahead_mask)
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(x + attn1)
        attn2 = self.mha2(out1, enc_output, enc_output, padding_mask)
        attn2 = self.dropout2(attn2, training=training)
        out2 = self.layernorm2(out1 + attn2)
        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        return self.layernorm3(out2 + ffn_output)

    def get_config(self):
        config = super(DecoderLayer, self).get_config()
        config.update({
            'd_model': self.mha1.d_model,
            'num_heads': self.mha1.num_heads,
            'ff_dim': self.ffn.layers[0].units,
            'dropout_rate': self.dropout1.rate
        })
        return config

# Inference function for LSTM and Attention models
@tf.function(reduce_retracing=True)
def predict_step(model, inputs):
    return model(inputs, training=False)

def generate_title_lstm_attention(model, article_sequence, target_tokenizer, max_title_len):
    decoder_input = np.zeros((1, max_title_len - 1))
    title = []
    vocab_size = min(len(target_tokenizer.word_index) + 1, MAX_VOCAB_SIZE)

    for i in range(max_title_len - 1):
        predictions = predict_step(model, [article_sequence, decoder_input])
        predicted_id = np.argmax(predictions[0, i, :])

        # Debug: Cetak predicted_id untuk memeriksa
        print(f"Step {i}, Predicted ID: {predicted_id}")

        # Hentikan jika predicted_id tidak valid atau di luar kosakata
        if predicted_id == 0 or predicted_id >= vocab_size:
            break

        # Cari kata yang sesuai dengan predicted_id
        for word, index in target_tokenizer.word_index.items():
            if index == predicted_id:
                title.append(word)
                break
        else:
            # Jika tidak ditemukan kata, lanjutkan ke langkah berikutnya
            continue

        decoder_input[0, i] = predicted_id

    return ' '.join(title) if title else "No title generated"

# Inference function for Transformer model
def generate_title_transformer(model, article_sequence, target_tokenizer, max_title_len):
    decoder_input = np.zeros((1, max_title_len - 1))
    title = []
    vocab_size = min(len(target_tokenizer.word_index) + 1, MAX_VOCAB_SIZE)

    for i in range(max_title_len - 1):
        predictions = predict_step(model, [article_sequence, decoder_input])
        predicted_id = np.argmax(predictions[0, i, :])

        # Debug: Cetak predicted_id untuk memeriksa
        print(f"Transformer Step {i}, Predicted ID: {predicted_id}")

        # Hentikan jika predicted_id tidak valid atau di luar kosakata
        if predicted_id == 0 or predicted_id >= vocab_size:
            break

        # Cari kata yang sesuai dengan predicted_id
        for word, index in target_tokenizer.word_index.items():
            if index == predicted_id:
                title.append(word)
                break
        else:
            continue

        decoder_input[0, i] = predicted_id

    return ' '.join(title) if title else "No title generated"

# Main function to generate titles
def main(article_text):
    # Load tokenizers
    try:
        with open('input_tokenizer.pkl', 'rb') as f:
            input_tokenizer = pickle.load(f)
        with open('target_tokenizer.pkl', 'rb') as f:
            target_tokenizer = pickle.load(f)
    except FileNotFoundError:
        print("Error: Tokenizer files (input_tokenizer.pkl, target_tokenizer.pkl) not found.")
        return

    # Preprocess article
    article_sequence = preprocess_article(article_text, input_tokenizer)

    # Load and compile models
    try:
        lstm_model = tf.keras.models.load_model('lstm_encoder_decoder_model.h5')
        lstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        print("LSTM model loaded and compiled.")
    except Exception as e:
        print(f"Error loading LSTM model: {e}")
        lstm_model = None

    try:
        attention_model = tf.keras.models.load_model('encoder_decoder_attention_model.h5')
        attention_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        print("Attention model loaded and compiled.")
    except Exception as e:
        print(f"Error loading Attention model: {e}")
        attention_model = None

    try:
        transformer_model = tf.keras.models.load_model(
            'transformer_model.h5',
            custom_objects={
                'MultiHeadAttention': MultiHeadAttention,
                'EncoderLayer': EncoderLayer,
                'DecoderLayer': DecoderLayer
            }
        )
        transformer_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        print("Transformer model loaded and compiled.")
    except Exception as e:
        print(f"Error loading Transformer model: {e}")
        transformer_model = None

    # Generate titles
    print("\nGenerated Titles:")
    print("-------------------")

    if lstm_model:
        lstm_title = generate_title_lstm_attention(lstm_model, article_sequence, target_tokenizer, MAX_TITLE_LEN)
        print(f"LSTM Encoder-Decoder: {lstm_title}")
    else:
        print("LSTM Encoder-Decoder: Not available due to loading error")

    if attention_model:
        attention_title = generate_title_lstm_attention(attention_model, article_sequence, target_tokenizer, MAX_TITLE_LEN)
        print(f"Encoder-Decoder with Attention: {attention_title}")
    else:
        print("Encoder-Decoder with Attention: Not available due to loading error")

    if transformer_model:
        transformer_title = generate_title_transformer(transformer_model, article_sequence, target_tokenizer, MAX_TITLE_LEN)
        print(f"Transformer: {transformer_title}")
    else:
        print("Transformer: Not available due to loading error")

if __name__ == "__main__":
    # Artikel yang diberikan
    article = """
    The global economy is showing signs of recovery after a challenging year. Stock markets in major economies have risen by 10% in the last quarter, driven by strong corporate earnings and optimism about vaccine distribution. However, concerns remain about inflation and supply chain disruptions, which could slow growth in the coming months.
    """

    print("Input Article:")
    print("-------------")
    print(article.strip())

    # Jalankan fungsi untuk menghasilkan judul
    main(article)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Input Article:
-------------
The global economy is showing signs of recovery after a challenging year. Stock markets in major economies have risen by 10% in the last quarter, driven by strong corporate earnings and optimism about vaccine distribution. However, concerns remain about inflation and supply chain disruptions, which could slow growth in the coming months.


LSTM model loaded and compiled.


Attention model loaded and compiled.


Transformer model loaded and compiled.

Generated Titles:
-------------------
Step 0, Predicted ID: 0
LSTM Encoder-Decoder: No title generated
Step 0, Predicted ID: 0
Encoder-Decoder with Attention: No title generated
Transformer Step 0, Predicted ID: 0
Transformer: No title generated
